# Implied IRR Analysis: Sauder Capital Markets Challenge
**Objective:** Calculate the Implied Levered IRR based on Bloomberg Consensus forecasts and market-aligned exit multiples.

### Methodology
1. **FCFE Derivation:** We adjust Consensus FCFF for tax-affected interest expense to derive Free Cash Flow to Equity (FCFE), assuming a constant debt structure (Refinancing Scenario).
2. **Terminal Value:** Calculated using 'Peer Group Median' or 'Forward P/E' multiples consistent with the 2030 forecast horizon.
3. **Solver:** We solve for the discount rate (IRR) that equates the current Market Cap to the PV of future equity cash flows.

In [1]:
import numpy as np
import numpy_financial as npf
import pandas as pd
import os

def compute_irr(cash_flows):
    """
    Returns IRR as decimal (e.g. 0.14 == 14%).
    """
    irr = npf.irr(cash_flows)
    if irr is None or np.isnan(irr):
        return 0.0
    return irr

def save_summary_md(filename, stock, horizon, equity_val, exit_pe, implied_irr, irr_stream, fcfe_stream, tv):
    """
    Saves the key analysis outputs to a clean Markdown file.
    """
    moic = sum(irr_stream[1:]) / abs(irr_stream[0])
    
    with open(filename, "w") as f:
        f.write(f"# Implied IRR Analysis: {stock.upper()}\n")
        f.write(f"**Date:** December 2025 | **Horizon:** {horizon} Years\n\n")
        
        f.write("## 1. Valuation Summary\n")
        f.write(f"- **Implied Levered IRR:** `{implied_irr:.2%}`\n")
        f.write(f"- **Exit Multiple (P/FCF):** {exit_pe:.2f}x\n")
        f.write(f"- **Gross Multiple of Money (MOIC):** {moic:.2f}x\n\n")
        
        f.write("## 2. Key Assumptions\n")
        f.write("| Metric | Value |\n")
        f.write("| :--- | :--- |\n")
        f.write(f"| Initial Equity Value | {equity_val:,.0f} |\n")
        f.write(f"| Terminal Value (Yr {horizon}) | {tv:,.0f} |\n")
        f.write(f"| Avg FCFE (Forecast) | {np.mean(fcfe_stream):,.0f} |\n\n")

        f.write("## 3. Cash Flow Stream (for IRR)\n")
        f.write("| Year | Cash Flow (FCFE) |\n")
        f.write("| :--- | :---: |\n")
        f.write(f"| 0 (Investment) | ({abs(irr_stream[0]):,.0f}) |\n")
        
        # Intermediate years
        for i, cf in enumerate(irr_stream[1:-1], 1):
            f.write(f"| {i} | {cf:,.0f} |\n")
            
        # Final year
        f.write(f"| {horizon} (Incl. Exit) | {irr_stream[-1]:,.0f} |\n")
    
    print(f"\n[SUCCESS] Summary saved to: {os.path.abspath(filename)}")

### 1. Bloomberg Inputs
**Instructions:** Update the dictionaries below with live data from Bloomberg (`EE`, `RV`, `IS`).

In [2]:
# FCFF consensus (unlevered FCF), in absolute currency units
fcff_dict = {
    "nike":   [2_635.2e6, 3_765.5e6, 4_572.1e6, 4_965.6e6, 5_502.7e6],
    "lulu":   [1_152.6e6, 1_245.6e6, 1_405.3e6, 1_400.1e6, 1_503.7e6],
    "atz_cn": [ 384.9e6,   601.5e6,   679.1e6,   811.1e6,   956.7e6],
}

# Current Equity Value (Market Cap), same currency as FCFF
equity_value_dict = {
    "nike":   88_692_081_760.00,
    "lulu":   24_681_182_826.00,
    "atz_cn": 13_432_629_145.00,
}

# Interest expense (LTM) and effective tax rate (%)
debt_params = {
    "nike":   {"interest_exp": -67.0e6,  "tax_rate": 17.14},  
    "lulu":   {"interest_exp": 0.0,     "tax_rate": 29.56},  
    "atz_cn": {"interest_exp": 69.6e6,  "tax_rate": 28.46},  
}

# Exit P/FCF multiples from median of RV
exit_multiples = {
    "nike":   26.08,
    "lulu":   24.51,
    "atz_cn": 22.36,
}

### 2. Run Analysis
Select the stock ticker below to run the calculation.

In [3]:
# -----------------------------
# CONTROLS
# -----------------------------
STOCK = "lulu"        # Options: "nike", "lulu", or "atz_cn"
HORIZON_YEARS = 5

print(f"Running Analysis for: {STOCK.upper()}")

# -----------------------------
# FCFE & TERMINAL VALUE CALCULATION
# -----------------------------

fcff_stream = np.array(fcff_dict[STOCK][:HORIZON_YEARS], dtype=float)

params = debt_params[STOCK]
tax = params["tax_rate"] / 100.0               
interest = params["interest_exp"]              
after_tax_interest = interest * (1.0 - tax)    

# Constant net debt: FCFE = FCFF - after-tax interest
fcfe_stream = fcff_stream - after_tax_interest

exit_pe = exit_multiples[STOCK]
final_year_fcfe = fcfe_stream[-1]
terminal_value = final_year_fcfe * exit_pe

# -----------------------------
# IRR STREAM CONSTRUCTION
# -----------------------------

equity_investment = -equity_value_dict[STOCK]

# Cash-flow stream: t=0 outlay + HORIZON_YEARS FCFE (with TV in final year)
irr_stream = [equity_investment] + list(fcfe_stream[:-1]) + [fcfe_stream[-1] + terminal_value]
implied_irr = compute_irr(irr_stream)

# -----------------------------
# OUTPUT & VISUALIZATION
# -----------------------------

print(f"\n=== {STOCK.upper()} VALUATION RESULTS ===")
print(f"Implied Levered IRR: {implied_irr:.2%}")
print(f"Exit Multiple:       {exit_pe:.2f}x")
print(f"Terminal Value:      {terminal_value:,.0f}")

# Create a nice dataframe for display
df_output = pd.DataFrame({
    'Year': range(0, HORIZON_YEARS + 1),
    'Cash Flow Type': ['Initial Investment'] + ['FCFE'] * (HORIZON_YEARS - 1) + ['FCFE + Terminal Value'],
    'Amount': irr_stream
})
df_output['Amount'] = df_output['Amount'].apply(lambda x: f"{x:,.0f}")
df_output

Running Analysis for: LULU

=== LULU VALUATION RESULTS ===
Implied Levered IRR: 12.97%
Exit Multiple:       24.51x
Terminal Value:      36,855,687,000


,Year,Cash Flow Type,Amount
0,0,Initial Investment,"-24,681,182,826"
1,1,FCFE,"1,152,600,000"
2,2,FCFE,"1,245,600,000"
3,3,FCFE,"1,405,300,000"
4,4,FCFE,"1,400,100,000"
5,5,FCFE + Terminal Value,"38,359,387,000"


In [4]:
# Save to Markdown File
output_filename = f"./irr_summaries/{STOCK}_IRR_Summary.md"
save_summary_md(
    output_filename, 
    STOCK, 
    HORIZON_YEARS, 
    -equity_investment, 
    exit_pe, 
    implied_irr, 
    irr_stream, 
    fcfe_stream, 
    terminal_value
)


[SUCCESS] Summary saved to: c:\Users\skurono\Desktop\scmc\scmc-2025\irr_summaries\lulu_IRR_Summary.md
